In [1]:
import pandas as pd
import bioframe

In [2]:
eigenvector = pd.read_table("/exports/igmm/eddie/wendy-lab/ilia/interactions_screen_quaich/results/eigenvectors/Hsieh_microC_50000_eigenvectors.cis.vecs.tsv",
                            sep="\t")

In [3]:
eigenvector = eigenvector.dropna()

In [4]:
A_compartments = bioframe.merge(eigenvector.loc[eigenvector["E1"]>0,:])
A_compartments["start2"] = A_compartments["end"]
A_compartments["end2"] = A_compartments["end"]+1
A_compartments["strand"] = "+"

In [5]:
B_compartments = bioframe.merge(eigenvector.loc[eigenvector["E1"]<0,:])
B_compartments["start2"] = B_compartments["end"]
B_compartments["end2"] = B_compartments["end"]+1
B_compartments["strand"] = "-"

In [6]:
compartment_switches = pd.concat([A_compartments[A_compartments["end"].isin(B_compartments["start"])], 
                                  B_compartments[B_compartments["end"].isin(A_compartments["start"])]]).sort_values(["chrom", "start2"])
compartment_switches[["chrom", 
                      "start2",
                      "end2",
                      "strand", 
                      "strand", 
                      "strand"]].to_csv("Hsieh_microC_compartment_boundaries_A_plus_B_minus_mm10.bed",
                                        sep="\t", index=False, header=False)                                                              

In [8]:
#Discard A compartments smaller than 100kb
A_compartments.loc[A_compartments["n_intervals"] >=2,
                  ["chrom", "start", "end"]].to_csv("Hsieh_microC_A_compartment_regions_100kb_mm10.bed", 
                                                    index=False, header=False, sep="\t")

In [9]:
!bedtools intersect -a mm10.refGene_exon1_TSS_noRING1B.bed \
-b Hsieh_microC_A_compartment_regions_100kb_mm10.bed \
-wa | bedtools sort -i | bedtools closest -a stdin \
-b <(bedtools sort -i Hsieh_microC_compartment_boundaries_A_plus_B_minus_mm10.bed) \
-d | awk -v OFS="\t" '{if($13<50000) {print $1,$2,$3,$4,$5,$10}}' \
> TSS_noRING1B_A_within_50kb_compartmentswitch_signAB_mm10.bed

In [10]:
!wc -l TSS_noRING1B_A_within_50kb_compartmentswitch_signAB_mm10.bed

1139 TSS_noRING1B_A_within_50kb_compartmentswitch_signAB_mm10.bed


In [16]:
!bedtools intersect -a ESC_4SU_refGene_exon1_quartile_4_TSS_mm10_noRING1B.bed \
-b Hsieh_microC_A_compartment_regions_100kb_mm10.bed \
-wa | bedtools sort -i | bedtools closest -a stdin \
-b <(bedtools sort -i Hsieh_microC_compartment_boundaries_A_plus_B_minus_mm10.bed) \
-d | awk -v OFS="\t" '{if($13>100000) {print $1,$2,$3,$4,$5,$10}}' \
| shuf -n 1139 > TSS_Q4_noRING1B_A_atleast_100kb_from_compartmentswitch_signAB_mm10.bed